# `Основные операторы PostgreSQL`

<img src='data/img/pglogo.png' width=700>

____

### <a id=0>Содержание</a>
- [Исходные данные](#1)
- [Задача](#2)
- [Решения](#2)


___
## <center> <a id=1>Исходные данные</a>

Дано два csv-файла с данными о клиентах :

- `customer.csv`

|Поле |Описание |
|:--|:--|
|customer_id  |id клиента |
| first_name  | 	имя клиента |
| last_name  | 	фамилия клиента |
| gender  |пол  |
|  DOB | 	дата рождения |
| job_title  | профессия |
| job_industry_category |	сфера деятельности  |
| wealth_segment  |	сегмент благосостояния  |
|deceased_indicator   | 	флаг актуального клиента |
| owns_car | флаг наличия автомобиля |
|address   | 	адрес проживания |
| postcode  |	почтовый индекс  |
| state  | штат |
|country   | страна проживания |
| property_valuation |  оценка имущества|

- `transaction.csv`

|Поле |Описание |
|:--|:--|
|transaction_id | id транзакции|
|product_id | 	id продукта|
|customer_id | id клиента|
|transaction_date |	дата транзакции |
|online_order |флаг онлайн-заказа |
| order_status| 	статус транзакции|
|brand | бренд|
| product_line| 	линейка продуктов|
| product_class|	класс продукта |
|product_size | 	размер продукта|
|list_price |	цена |
|standard_cost |	стандартная стоимость |


___
## <center> <a id=2>Задача</a>

$\boxed{1}$ Создать таблицы со следующими структурами и загрузить данные из csv-файлов

$\boxed{2}$ Выполнить следующие запросы :

- Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

-  Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.
- Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.
- Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services.
- Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.
- Вывести всех клиентов, у которых нет транзакций.
-  Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.
- Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

___
## <center> <a id=3>Решение</a>

___
###  <a id=3>Создание таблиц</a>

In [ ]:
create table customer (
"customer_id" int4,
"first_name" varchar(50),
"last_name" varchar(50),
"gender" varchar(30),
"DOB" varchar(50),
"job_title" varchar(50) ,
"job_industry_category" varchar(50) ,
"wealth_segment"  varchar(50),
"deceased_indicator"  varchar(50),
"owns_car"  varchar(30),
"address"  varchar(50),
"postcode"  varchar(30),
"state"  varchar(30),
"country"  varchar(30),
"property_valuation"  int4
);

create table transaction (
"transaction_id"  int4,
"product_id"  int4,
"customer_id"  int4,
"transaction_date"  varchar(30),
"online_order"  varchar(30),
"order_status"  varchar(30),
"brand"  varchar(30),
"product_line"  varchar(30),
"product_class"  varchar(30),
"product_size"  varchar(30),
"list_price"  float4,
"standard_cost"  float4
);

___
### <a id=4>Выполнение запросов</a>

- Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

In [ ]:
select distinct brand from transaction
where standard_cost > 1500;

- Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

In [ ]:
select * from transaction
where transaction_date in 
(select transaction_date from transaction where transaction_date ::date between '01.04.2017' and '09.04.2017' ) 
and order_status = 'Approved'

- Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [ ]:
select distinct job_title from customer
where (job_industry_category = 'IT' or job_industry_category = 'Financial Services')
and job_title like 'Senior%'

- Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

In [ ]:
select distinct brand from transaction
where customer_id in
(select customer_id from customer
where job_industry_category = 'Financial Services')

- Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [ ]:
select * from customer
where customer.customer_id in 
(select customer_id from transaction
where (brand = 'Giant Bicycles' or brand = 'Norco Bicycles' or brand =  'Trek Bicycles') and online_order = 'True' )
limit 10

- Вывести всех клиентов, у которых нет транзакций.

Через `вложенный запрос` :

In [ ]:
select * from customer
where customer_id not in
(select distinct customer_id from transaction )

Через `left join` :

In [ ]:
select * from customer c 
left join "transaction" t 
on c.customer_id = t.customer_id 
where t.customer_id is null 

- Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

Через `вложенный запрос` :

In [ ]:
select * from customer
where job_industry_category = 'IT'
and customer_id in
(select  customer_id from transaction
where  standard_cost in (select  standard_cost from transaction
where standard_cost notnull 
order by standard_cost desc
limit 1) )

Через `left join` :

In [ ]:
select * from customer c 
left join "transaction" t 
on c.customer_id = t.customer_id 
where t.standard_cost = (select  standard_cost from transaction
where standard_cost notnull 
order by standard_cost desc
limit 1) and c.job_industry_category = 'IT'

- Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [ ]:
select  distinct c.*  from customer c 
left join "transaction" t 
on  c.customer_id = t.customer_id 
where  (c.job_industry_category = 'IT' or c.job_industry_category = 'Health')
and t.order_status = 'Approved' 
and t.transaction_date ::date between '07.07.2017' and '17.07.2017'